In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs

from scsavailability import features as feat, model as md, plotting as pt

2020-12-08 14:57:57,139 - NumExpr defaulting to 8 threads.


In [2]:
path = '../data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201123.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-2511.csv')
fa = pd.read_csv(path + 'scs_tote_matched.csv')

In [4]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=2300811527072) was called 1 times


In [5]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2300811529520) was called 1 times


In [6]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=2300811528800) was called 1 times
Running: Function "load_module_lookup" (id=2300811529952) was called 1 times
duplicated location faults removed - max duration kept


In [7]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

Time shifted by 0m


In [8]:
fa_sel = feat.fault_select(fa_floor, select_level = 'Tote Colour', selection = ['Blue','Both'])

fa_agg = feat.faults_aggregate(fa_sel,fault_agg_level='code', agg_type = 'count')

Running: Function "faults_aggregate" (id=2300811529664) was called 1 times


In [9]:
av,at = feat.av_at_select(av, at, remove_high_AT = False)

av_agg = feat.aggregate_availability(av, agg_level = 'Module')
at_agg = feat.aggregate_totes(at, agg_level = 'Module')

Running: Function "aggregate_availability" (id=2300811526496) was called 1 times


In [10]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg, target = 'Blue Tote Loss',faults=True, totes = True, agg_level = 'Module')

Datasets merged


In [11]:
df = df[df['TOTES']<60].reset_index(drop=True)

In [12]:
X,y = md.gen_feat_var(df)

In [39]:
X_train, X_test, y_train, y_test = md.split(X,y,test_size=0.3,random_state=101)

#RF_mdl,predictions_RF=md.run_RF_model(X_train, X_test, y_train, y_test,num_trees=100, criterion = 'mse', max_depth=None, dtree=False)

#X_sel_rf = md.select_features(X, X_train, y_train, RF_mdl, thres = np.inf(), max_feat=1000)

Linear_mdl, predictions_LM, Coeff =md.run_LR_model(X_train, X_test, y_train, y_test)

#X_sel_lm = md.select_features(X, X_train, y_train, Linear_mdl, thres = np.inf(), max_feat=1000)

cv_R2 = md.cross_validate_r2(Linear_mdl, X, y, n_folds = 10, shuffle = True, random_state = 101)

# cv_R2 = md.cross_validate_r2(RF_mdl, X, y, n_folds = 10, shuffle = True, random_state = 101)

Feature Coefficient Ranking: 
 
    Coefficients      Feature
0  7.512939e+09          C42
1  6.911737e+09  C0704STA149
2  5.047681e+09  C0703STA105
3  4.367010e+09  C0901STA005
4  4.246942e+09  C0704STA126
5  4.211062e+09  C0502STA051
6  4.067609e+09  C0704STA130
7  3.935913e+09  C1502STA034
8  3.904325e+09  C1204STA139
9  3.848134e+09  C1602STA033

Regression Metrics: 
 
           LM Metrics
MAE         0.081005
MSE         0.011215
RMSE        0.105902
MAPE%            inf
ACC%            -inf
R2_Train    0.720426
R2_Pred     0.580434

Cross Validation Scores LinearRegression(): 
 
        R2 Scores
1       0.646174
2       0.511125
3       0.559401
4       0.685725
5       0.585456
6       0.665339
7       0.638576
8       0.672617
9       0.696947
10      0.647503
Mean    0.630886
STD     0.056908


In [42]:
cv_R2

0.6308862407924811

In [36]:
from datetime import datetime

In [37]:
from datetime import datetime

today = datetime.now()

# dd/mm/YY
d1 = today.strftime("%Y%m%d%H%M%S")

In [38]:
d1

'20201208150832'